# SquadBeyond 自動化ツール

## 使い方
1. **セル1** を実行 → セットアップ（初回のみ1〜2分かかります）
2. **セル2** を実行 → Google Driveからアカウント情報を読み込み
3. **セル3** を実行 → 自動化ロジックの準備
4. **セル4** を実行 → アカウント選択・CSVファイルアップロード・実行モード選択
5. **セル5** を実行 → 処理開始

**上から順番に全てのセルを実行してください。**

In [ ]:
#@title **セル1: セットアップ**（初回は1〜2分かかります）

# Google Chrome 安定版を直接インストール（Colabのchromiumはsnap版で動かないため）
!apt-get update -qq > /dev/null
!apt-get install -qq libu2f-udev libvulkan1 > /dev/null 2>&1
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb > /dev/null 2>&1
!apt-get -f install -qq > /dev/null 2>&1

# 必要なパッケージのインストール
!pip install -q selenium pandas chromedriver-autoinstaller

# ライブラリの読み込み
import time
import os
import io
import math
import logging
import concurrent.futures
import threading
import pandas as pd
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# ChromeDriverをChromeバージョンに合わせて自動インストール
chromedriver_autoinstaller.install()

# urllib3の警告ログを非表示にする
logging.getLogger('urllib3').setLevel(logging.ERROR)
logging.getLogger('selenium').setLevel(logging.ERROR)

print("✅ セットアップ完了")

In [ ]:
#@title **セル2: アカウント情報の読み込み**
#@markdown Googleアカウントでログインしてください（ポップアップが出ます）

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io as _io

auth.authenticate_user()

service = build('drive', 'v3')
request = service.files().get_media(fileId='12cYadGtLVyV9mJwsYNH2o9xalmJmHi1r')
fh = _io.FileIO('accounts.csv', 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    status, done = downloader.next_chunk()

df_acc = pd.read_csv('accounts.csv')
ACCOUNTS = {}
for i, row in df_acc.iterrows():
    key = str(i + 1)
    ACCOUNTS[key] = {
        "name": str(row["name"]),
        "id": str(row["id"]),
        "pass": str(row["pass"]),
    }
print("✅ アカウント情報を読み込みました")

In [ ]:
#@title **セル3: 自動化ロジックの準備**

# === 定数 ===
LOGIN_URL = 'https://app.squadbeyond.com/'
BASE_URL = 'https://app.squadbeyond.com/'
MAX_WORKERS = 3

# === ログアウト検知用フラグ（全ワーカー共有） ===
_logged_out = threading.Event()

# === ポート管理（ワーカーごとに異なるポートを割り当て） ===
_port_counter = threading.Lock()
_next_port = [9222]

def _get_next_port():
    with _port_counter:
        port = _next_port[0]
        _next_port[0] += 1
        return port

# === Chrome設定 ===
def get_chrome_options(port=None):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-software-rasterizer')
    if port:
        options.add_argument(f'--remote-debugging-port={port}')
    return options

# === Chromeドライバーを生成 ===
def create_driver():
    port = _get_next_port()
    options = get_chrome_options(port)
    return webdriver.Chrome(options=options)

# === ログアウトされたかチェック ===
def is_logged_out(driver):
    """「ログインセッションの有効期限が過ぎました」のポップアップを検知"""
    try:
        page_text = driver.execute_script("return document.body.innerText || '';")
        if 'ログインセッションの有効期限が過ぎました' in page_text:
            return True
        if 'ログインページへ' in page_text and 'トップページからログイン' in page_text:
            return True
    except:
        pass
    return False

# === ログインしてセッションCookieを取得 ===
def get_session_cookies(account_info, log_func):
    log_func('ログイン中...')

    driver = create_driver()
    wait = WebDriverWait(driver, 20)
    cookies = []

    try:
        driver.get(LOGIN_URL)
        time.sleep(2)

        email_xpath = "//input[@id=':r1:'] | //input[@name='email']"
        email_input = wait.until(EC.element_to_be_clickable((By.XPATH, email_xpath)))
        email_input.clear()
        email_input.send_keys(account_info['id'])

        pass_xpath = "//input[@id=':r2:'] | //input[@type='password']"
        pass_input = wait.until(EC.element_to_be_clickable((By.XPATH, pass_xpath)))
        pass_input.clear()
        pass_input.send_keys(account_info['pass'])

        btn1_xpath = "//button[@data-trackid='sign-in-form-login-button']"
        wait.until(EC.element_to_be_clickable((By.XPATH, btn1_xpath))).click()
        time.sleep(3)

        btn2_xpath = "//button[text()='ログイン' and contains(@class, 'MuiButton-sizeSmall')]"
        wait.until(EC.element_to_be_clickable((By.XPATH, btn2_xpath))).click()
        time.sleep(5)

        cookies = driver.get_cookies()
        log_func('✅ ログイン成功')

    except Exception as e:
        log_func(f'❌ ログインに失敗しました')
    finally:
        driver.quit()

    return cookies

# === 自動化ワーカー ===
def run_automation_worker(worker_id, df_subset, cookies, log_func, reset_rates):
    if _logged_out.is_set():
        return

    driver = create_driver()
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(BASE_URL)

        if not cookies:
            driver.quit()
            return

        for cookie in cookies:
            if 'expiry' in cookie:
                cookie['expiry'] = int(cookie['expiry'])
            driver.add_cookie(cookie)

        driver.refresh()
        time.sleep(3)

        for index, row in df_subset.iterrows():
            if _logged_out.is_set():
                return

            col1_url = str(row.iloc[0]) if pd.notna(row.iloc[0]) else ''
            col2_ver = str(row.iloc[1]) if pd.notna(row.iloc[1]) else ''
            col3_chk = str(row.iloc[2]) if pd.notna(row.iloc[2]) else ''
            col4_key = str(row.iloc[3]) if pd.notna(row.iloc[3]) else ''
            col5_lbl = str(row.iloc[4]) if pd.notna(row.iloc[4]) else ''
            col7_url = str(row.iloc[6]) if pd.notna(row.iloc[6]) else ''

            if not col1_url or col1_url == 'nan' or col1_url == '':
                continue

            driver.get(col1_url)
            time.sleep(2)

            # ページ遷移後にログアウトチェック
            if is_logged_out(driver):
                _logged_out.set()
                log_func('🚫 ログアウトされました。処理を停止します。')
                return

            try:
                xpath_ver = f"//span[@data-test='ArticleList-Memo' and contains(text(), '{col2_ver}')]"
                wait.until(EC.element_to_be_clickable((By.XPATH, xpath_ver))).click()
            except Exception:
                if is_logged_out(driver):
                    _logged_out.set()
                    log_func('🚫 ログアウトされました。処理を停止します。')
                    return
                continue

            try:
                menu_xpath = '//*[@id="root"]/div[2]/div/div[4]/div[2]/div/div[2]/div/div[2]/div/div[2]/button'
                wait.until(EC.element_to_be_clickable((By.XPATH, menu_xpath))).click()
                time.sleep(0.5)
                clone_menu_xpath = "//span[contains(text(), '別のbeyondページに複製')]"
                wait.until(EC.element_to_be_clickable((By.XPATH, clone_menu_xpath))).click()
            except:
                if is_logged_out(driver):
                    _logged_out.set()
                    log_func('🚫 ログアウトされました。処理を停止します。')
                    return
                driver.execute_script("""
                var spans = document.querySelectorAll('span');
                for (var i = 0; i < spans.length; i++) {
                    if (spans[i].textContent.includes('別のbeyondページに複製')) {
                        spans[i].click();
                        break;
                    }
                }
                """)
            time.sleep(1)

            try:
                select_xpath = "//select[@data-test='DuplicateToOtherModal-DestinationAbTestUidSelect']"
                select_elem = wait.until(EC.presence_of_element_located((By.XPATH, select_xpath)))
                Select(select_elem).select_by_visible_text(col5_lbl)
                btn_xpath = "//div[contains(text(), '複製する')]"
                wait.until(EC.element_to_be_clickable((By.XPATH, btn_xpath))).click()
            except Exception as e:
                pass

            time.sleep(3)

            if is_logged_out(driver):
                _logged_out.set()
                log_func('🚫 ログアウトされました。処理を停止します。')
                return

            try:
                rate_xpath = f"//input[@value='{col4_key}']/following::div[@data-test='ArticleList-DeriveryUpRateForm'][1]"
                wait.until(EC.element_to_be_clickable((By.XPATH, rate_xpath))).click()
            except Exception as e:
                pass

            if reset_rates:
                js_rate_0 = """
                var others = document.querySelectorAll("div[data-test='ArticleList-Article'] input[data-test='ArticleList-DeriveryRateForm']");
                var setter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
                others.forEach(function(i){
                    setter.call(i, "0");
                    i.dispatchEvent(new Event('input', { bubbles: true }));
                });
                """
                driver.execute_script(js_rate_0)

            try:
                link_replace_xpath = '//*[@id="root"]/div[2]/div/div[4]/div[7]/div/div[4]/div/div/div/div/div/img'
                time.sleep(1)
                replace_icon = wait.until(EC.element_to_be_clickable((By.XPATH, link_replace_xpath)))
                replace_icon.click()
                time.sleep(1)
            except Exception as e:
                pass

            try:
                chk_xpath = f"//div[contains(text(), '{col3_chk}')]"
                chk_elem = wait.until(EC.element_to_be_clickable((By.XPATH, chk_xpath)))
                chk_elem.click()
            except Exception as e:
                pass

            try:
                input_xpath = "//input[@placeholder='新規のリンクを入力']"
                input_elem = wait.until(EC.presence_of_element_located((By.XPATH, input_xpath)))
                driver.execute_script("""
                    var element = arguments[0];
                    var value = arguments[1];
                    var setter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
                    setter.call(element, value);
                    element.dispatchEvent(new Event('input', { bubbles: true }));
                """, input_elem, col7_url)
            except Exception as e:
                pass
            time.sleep(1)

            try:
                replace_btn_xpath = '//*[@id="root"]/div[2]/div/div[4]/div[7]/div/div[4]/div/div[2]/div/div/div[2]/div[5]/div/div[2]'
                wait.until(EC.element_to_be_clickable((By.XPATH, replace_btn_xpath))).click()
            except:
                driver.execute_script("var b=Array.from(document.querySelectorAll('div')).find(e=>e.textContent=='置換');if(b)b.click();")
            time.sleep(1)

            driver.refresh()
            time.sleep(2)

            if is_logged_out(driver):
                _logged_out.set()
                log_func('🚫 ログアウトされました。処理を停止します。')
                return

            try:
                update_btn_xpath = "//button[text()='更新']"
                wait.until(EC.element_to_be_clickable((By.XPATH, update_btn_xpath))).click()
                target_class_fragment = 'css-1g290g6'
                success_condition_xpath = f"//button[text()='更新' and contains(@class, '{target_class_fragment}')]"
                wait.until(EC.presence_of_element_located((By.XPATH, success_condition_xpath)))
                time.sleep(0.5)
            except Exception as e:
                try:
                    driver.execute_script("""
                    var btns = document.querySelectorAll("button");
                    var target = Array.from(btns).find(b => b.textContent.trim() === "更新");
                    if(target){ target.click(); }
                    """)
                    time.sleep(2)
                except:
                    pass

            try:
                current_url = driver.current_url
                if '#' in current_url:
                    base_url = current_url.split('#')[0]
                else:
                    base_url = current_url.rstrip('/').rsplit('/', 1)[0]
                target_url = base_url + '/exit_popups'
                driver.get(target_url)
                time.sleep(2)
            except Exception as e:
                pass

            try:
                target_xpath = f"//h6[contains(text(), '{col4_key}')]"
                wait.until(EC.element_to_be_clickable((By.XPATH, target_xpath))).click()
            except:
                pass

            try:
                base_xpath = "//span[@aria-label='ONにすると選択中のVersionにポップアップ/バナーが紐づきます']"
                wait.until(EC.presence_of_element_located((By.XPATH, base_xpath)))
                driver.execute_script("""
                var parentSpan = document.querySelector("span[aria-label='ONにすると選択中のVersionにポップアップ/バナーが紐づきます']");
                var input = parentSpan.querySelector("input");
                if(input && !input.checked){
                    input.click();
                }
                """)
                on_state_xpath = base_xpath + "[contains(@class, 'Mui-checked')]"
                wait.until(EC.presence_of_element_located((By.XPATH, on_state_xpath)))
            except Exception as e:
                pass

            try:
                url_xpath = "//input[@name='urlControl']"
                url_input = wait.until(EC.element_to_be_clickable((By.XPATH, url_xpath)))
                current_val = url_input.get_attribute('value')
                if current_val != col7_url:
                    driver.execute_script("""
                        var element = arguments[0];
                        var value = arguments[1];
                        var setter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
                        setter.call(element, value);
                        element.dispatchEvent(new Event('input', { bubbles: true }));
                    """, url_input, col7_url)
                    time.sleep(1)
                    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='本番反映']"))).click()
            except Exception as e:
                pass

            log_func(f'✅ {index+1}件目 完了: {col1_url}')
            time.sleep(1)

    except Exception as e:
        if is_logged_out(driver):
            _logged_out.set()
            log_func('🚫 ログアウトされました。処理を停止します。')
        else:
            log_func(f'❌ エラーが発生しました')
    finally:
        driver.quit()

print('✅ 準備完了')

In [ ]:
#@title **セル4: 実行設定**

from google.colab import files

# アカウント一覧を表示（番号・名前・メールアドレス）
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print(' 番号  名前                    メールアドレス')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
for k, v in ACCOUNTS.items():
    print(f'  {k}    {v["name"]:<24}{v["id"]}')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print()

account_number = input('使用するアカウント番号を入力: ')
if account_number not in ACCOUNTS:
    print(f'❌ アカウント番号 {account_number} は存在しません')
else:
    selected_account = ACCOUNTS[account_number]
    print(f'→ {selected_account["name"]}（{selected_account["id"]}）を使用します')
    print()

    # CSVファイルをアップロード
    print('CSVファイルをアップロードしてください:')
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    csv_data = uploaded[filename].decode('utf-8')
    print(f'→ {filename} を読み込みました')
    print()

    mode = input('実行モード（1: 他を0%にする / 2: 割合変更なし）: ')
    reset_rates = (mode == '1')
    print(f'→ モード: {"他を0%にする" if reset_rates else "割合変更なし"}')
    print()
    print('✅ 設定完了。次のセルを実行してください。')

In [ ]:
#@title **セル5: 実行**

# ログアウトフラグをリセット
_logged_out.clear()

try:
    df = pd.read_csv(io.StringIO(csv_data), header=None)
except Exception as e:
    print(f'❌ CSVファイルの読み込みに失敗しました')
    df = None

if df is not None:
    print(f'処理対象: {len(df)} 件')
    print('実行中...')
    print()

    cookies = get_session_cookies(selected_account, print)

    if not cookies:
        print('❌ ログインに失敗しました')
    else:
        total_rows = len(df)
        chunk_size = math.ceil(total_rows / MAX_WORKERS)
        chunks = [df.iloc[i:i + chunk_size] for i in range(0, total_rows, chunk_size)]

        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = []
            for i, df_subset in enumerate(chunks):
                if i > 0:
                    time.sleep(5)
                futures.append(
                    executor.submit(run_automation_worker, i + 1, df_subset, cookies, print, reset_rates)
                )
            concurrent.futures.wait(futures)

        print()
        print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
        if _logged_out.is_set():
            print('🚫 セッションが切れたため途中で停止しました')
        else:
            print(f'✅ 全 {total_rows} 件の処理が完了しました')
        print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')